In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import pandas
import numpy

# Input data

In [2]:
data = pandas.read_csv('hits_1000.csv', index_col=False)
#data = data[data.event.values < 100]

data.head()

,event,particle,layer,iphi,x,y
0,0,0,5,39276,55.103343,-401.233874
1,0,6,5,22685,-381.682239,135.438799
2,0,3,3,6082,160.995866,139.460859
3,0,5,2,27787,-35.433651,-150.895515
4,0,5,1,15230,-19.627350,-82.702885


# Hough Transform with Tracks Classification

## Data Preparation

In [3]:
X = data[[u'event', u'layer', u'iphi', u'x', u'y']].values
y = data[u'particle'].values

## Track Pattern Recognition

In [4]:
from nearest_hit import NearestHit

nh = NearestHit(min_cos_value=0.9)

nh.fit(X, y)
y_reco = nh.predict(X)

## Quality metrics

In [5]:
from metrics import RecognitionQuality

rq = RecognitionQuality(track_eff_threshold=0.8, min_hits_per_track=4)
report_event, report_tracks = rq.calculate(X, y, y_reco)

In [6]:
report_event.mean(axis=0)

Event                       499.500000
ReconstructionEfficiency      0.786045
GhostRate                     0.187597
CloneRate                     0.000000
AvgTrackEfficiency            0.847078
dtype: float64